# DCGAN(Deep Convolutional Generative Adversarial Network)
- DCGAN의 구성 요소인 Generator와 Discriminator 구현
- GAN loss를 통한 적대적 학습
- CelebA dataset을 이용한 인물 사진 생성

## Prep

In [1]:
!pip install gdown -q # Google download 링크

In [ ]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# 코드 실행결과의 동일성을 위해 무작위 시드를 설정
manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # 결과 재현

device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

### 데이터 다운로드

In [ ]:
celebA = dset.CelebA(root='./data', split='train', download=True)

In [4]:
dataroot = "/kaggle/working/data/celeba" # 데이터셋의 경로
workers = 2 # dataloader에서 사용할 쓰레드 수
batch_size = 128
image_size = 64

nc = 3 # 이미지의 채널 수로, RGB 이미지이기 때문에 3으로 설정합니다.
nz = 100 # Latent space vector z의 크기 (예. 생성자의 입력값 크기)
ngf = 64 # 생성자를 통과하는 Feature map의 Channel depth
ndf = 64 # 구분자를 통과하는 Feature map의 Channel depth

num_epochs = 5
lr = 0.0002
beta1 = 0.5 # Adam 옵티마이저의 beta1 하이퍼파라미터

### Dataset & Dataloader

In [ ]:
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# dataloader
dataloader = DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# 학습 데이터 시각화
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()

### 네트워크 가중치 초기화 함수

In [6]:
# Generator, Discriminator 에 적용시킬 커스텀 가중치 초기화 함수
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

### Generator 네트워크 구현

In [7]:
# Generator(생성자) class

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # z -> 3차원 텐서
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # 첫 번째 Upsampling
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # 두 번쨰 Upsampling
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # 세 번쨰 Upsampling
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # 마지막 Upsampling
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
netG = Generator().to(device)

# 모든 가중치의 평균을 0( ``mean=0`` ), 분산을 0.02( ``stdev=0.02`` )로 초기화
netG.apply(weights_init)

# 모델의 구조를 출력합니다
print(netG)

In [ ]:
dummy_z = torch.rand(1, 100, 1, 1)
dummy_out_G = netG(dummy_z.to(device))
dummy_out_G.shape

### Discriminator 네트워크 구현

In [10]:
# Discriminator(판별자) class

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
netD = Discriminator().to(device)

# 모든 가중치의 평균을 0( ``mean=0`` ), 분산을 0.02( ``stdev=0.02`` )로 초기화
netD.apply(weights_init)

# 모델의 구조를 출력합니다
print(netD)

In [ ]:
dummy_img = torch.rand(1, 3, 64, 64)
dummy_out_D = netD(dummy_img.to(device))
dummy_out_D.shape

## 학습

### 학습 유틸리티 설정

In [18]:
# Discriminator loss
criterion = nn.BCELoss()

# Generator의 학습상태를 확인할 latent space vector 생성
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# 학습에 사용되는 참/거짓 레이블
real_label = 1.
fake_label = 0.

# G와 D에서 사용할 옵티마이저
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

### 학습 루프

In [ ]:
# 학습상태를 체크하기 위해 손실값들을 저장
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# 에폭(epoch) 반복
for epoch in range(num_epochs):
    # 한 에폭 내에서 배치 반복
    for i, data in enumerate(dataloader, 0):

        # Discriminator 신경망 업데이트: log(D(x)) + log(1 - D(G(z))) 최대화
        ## 진짜 데이터 학습
        netD.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label,
                           dtype=torch.float, device=device)

        output = netD(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward() # Discriminator update
        D_x = output.mean().item()

        ## 가짜 데이터 학습
        noise = torch.randn(b_size, nz, 1, 1, device=device) # Generator에 사용할 Latent space vector를 생성
        fake = netG(noise) # G를 이용해 가짜 이미지를 생성합니다
        label.fill_(fake_label)
        output = netD(fake.detach()).view(-1) # D를 이용하여 데이터의 진위 판별
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # 가짜 이미지와 진짜 이미지 모두에서 구한 손실값 합산
        # 이때 errD는 역전파에서 사용되지 않고, 이후 학습 상태를 리포팅(reporting)할 때 사용
        errD = errD_real + errD_fake
        optimizerD.step()

        # Generator 신경망 업데이트: log(D(G(z)))를 최대화
        netG.zero_grad()
        label.fill_(real_label)  # 생성자의 손실값을 구하기 위해 진짜 레이블 이용
        # Discriminator는 업데이트 되었지만, Generator는 업데이트되지 않은 상태
        # Discriminator에 다시 가짜 데이터를 통과 
        # D가 업데이트 되었기 때문에 앞선 손실값과 다른 값이 나옴
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # 훈련 상태를 출력
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # 이후 그래프를 그리기 위하여 손실값들을 저장
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # fixed_noise를 통과시킨 G의 출력값을 저장
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
# dataloader에서 실제 이미지 가져오기
real_batch = next(iter(dataloader))

# 진짜 이미지 출력
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# 가짜 이미지 출력
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()